In [ ]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3
from torchvision import datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader, Subset

import numpy as np
from scipy.stats import entropy


In [ ]:
def inception_score(imgs, cuda=True, batch_size=32, resize=False, splits=1):
    N = len(imgs)
    dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)

    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval();
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x, dim=1).data.cpu().numpy()

    preds = np.zeros((N, 1000))

    for i, batch in enumerate(dataloader):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)


In [ ]:
def inception_score(dataloader: DataLoader, cuda=True, resize=False, splits=1):
    N = len(dataloader.dataset)
    batch_size = dataloader.batch_size
    dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval();
    
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x, dim=1).data.cpu().numpy()

    preds = np.zeros((N, 1000))

    for i, batch in enumerate(dataloader):
        if i>3:
            break
        print(i)
        batch = batch[0]
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)


In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, orig):
        self.orig = orig

    def __getitem__(self, index):
        return self.orig[index][0]

    def __len__(self):
        return len(self.orig)

import torchvision.datasets as dset
import torchvision.transforms as transforms

cifar = dset.CIFAR10(root='data/', download=True,
                         transform=transforms.Compose([
                             transforms.Resize(32),
                             transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                         ])
)

print (inception_score(CustomDataset(cifar), cuda=True, batch_size=64, resize=True, splits=1))


In [ ]:

inception_score(input_data_synthetic)/ inception_score(input_real_images) 

In [ ]:
# configure image transformations
rm_transforms = []
rm_transforms += [transforms.ToTensor()]
rm_transforms += [transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]


# load real data
real_dataset = datasets.ImageFolder(root='data/Alzheimer_s Dataset/train',
                                   transform=transforms.Compose(rm_transforms))
real_dataloader = DataLoader(dataset=real_dataset, batch_size=32)

# load synth data
synth_dataset = datasets.ImageFolder(root='data/synth',
                                    transform=transforms.Compose(rm_transforms))
synth_dataloader = DataLoader(dataset=synth_dataset, batch_size=32)

In [ ]:
inception_score(synth_dataloader, resize=False, cuda=False)

In [ ]:
inception_score(real_dataloader)